In [60]:
import pandas as pd
from constants import *
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
fname = "data/ev_sales/BEV-PHEV-HEV-FCEV Sales By State for 2011-2021-EVAdoption.xlsx"
sample_df = pd.read_excel(fname, sheet_name = '2013')

In [48]:
start_year = 2011
end_year = 2019
read_cols_v1 = ["State", "FCEV Sales", "BEV Sales", "PHEV Sales"]
read_cols_v2 = ["State", "FCEV Sales","FCEV Market Share", "BEV Sales", "BEV Market Share", "PHEV Sales", "PHEV Market Share"]


df = pd.DataFrame(columns=read_cols_v2)
for year in range(start_year, end_year+1):
    read_cols = read_cols_v2 if year >=2013 else read_cols_v1
    filter_cols = ["{} {}".format(year, j) if i>0 else j for i, j in enumerate(read_cols)]
    t_df = pd.read_excel(fname, sheet_name="{}".format(year), usecols=filter_cols)
    t_df.columns = read_cols
    t_df['year'] = str(year)
    df = df.append(t_df)

C:\Users\harsh\AppData\Local\Temp\ipykernel_16040\2189963207.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(t_df)
C:\Users\harsh\AppData\Local\Temp\ipykernel_16040\2189963207.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(t_df)
C:\Users\harsh\AppData\Local\Temp\ipykernel_16040\2189963207.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(t_df)
C:\Users\harsh\AppData\Local\Temp\ipykernel_16040\2189963207.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(t_df)
C:\Users\harsh\AppData\Local\Temp\ipykernel_16040\2189963207.py:14: FutureWarning: The frame.append meth

In [49]:
df['State'] = df['State'].replace("District of Columbia", "District Of Columbia")

In [50]:
epa_region = pd.read_csv("data/states_and_counties.csv")
epa_region = epa_region[['State Name','State Code', "EPA Region"]]
epa_region = epa_region[epa_region['State Code'] != 'CC']
epa_region['State Code'] = epa_region['State Code'].apply(int)
epa_region = epa_region.drop_duplicates(['State Code'], keep='first')
df = df.merge(epa_region, left_on="State", right_on="State Name", how='left')

In [51]:
def get_max(x):
    x = max(x['BEV Sales']/x['BEV Market Share'], x['PHEV Sales']/x['PHEV Market Share'])
    x = math.ceil(x) if not pd.isna(x) else x
    return x

df['ev_sales'] =  df["FCEV Sales"] + df["BEV Sales"] +  df["PHEV Sales"] 
df['ev_market_share'] =  df["FCEV Market Share"] + df["BEV Market Share"] +  df["PHEV Market Share"] 
df['ev_sales'] =  df["FCEV Sales"] + df["BEV Sales"] +  df["PHEV Sales"] 
df['total'] = df.apply(get_max, axis=1)
df['nonev_sales'] = df['total'] - df['ev_sales']

In [52]:
df = df[~pd.isna(df['State Code'])]
df['year'] = df['year'].astype(int)
df['State Code'] = df['State Code'].astype(int)
df = df.rename(columns={'State Code': 'state_code'})

In [53]:
df.to_csv("data/ev_sales/ev_data_preprocessed.csv", index=False)

In [54]:
df

,State,FCEV Sales,FCEV Market Share,BEV Sales,BEV Market Share,PHEV Sales,PHEV Market Share,year,State Name,state_code,EPA Region,ev_sales,ev_market_share,total,nonev_sales
0,Alabama,0,NaN,18,NaN,55,NaN,2011,Alabama,1,4.0,73,NaN,NaN,NaN
1,Alaska,0,NaN,2,NaN,4,NaN,2011,Alaska,2,10.0,6,NaN,NaN,NaN
2,Arizona,0,NaN,355,NaN,102,NaN,2011,Arizona,4,9.0,457,NaN,NaN,NaN
3,Arkansas,0,NaN,2,NaN,20,NaN,2011,Arkansas,5,6.0,22,NaN,NaN,NaN
4,California,19,NaN,5062,NaN,1656,NaN,2011,California,6,9.0,6737,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,Virginia,0,0.0,5677,0.0153,1337,0.0036,2019,Virginia,51,3.0,7014,0.0189,371389.0,364375.0
463,Washington,0,0.0,10256,0.0359,1916,0.0067,2019,Washington,53,10.0,12172,0.0426,285971.0,273799.0
464,West Virginia,0,0.0,179,0.0035,71,0.0014,2019,West Virginia,54,3.0,250,0.0049,51143.0,50893.0
465,Wisconsin,0,0.0,1381,0.0057,581,0.0024,2019,Wisconsin,55,5.0,1962,0.0081,242281.0,240319.0


In [ ]:
df['']